<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fclassification/applications/classification/QQP%20Classification%20with%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quora Duplicate Question Pair detection with BERT

### Imports

In [0]:
!pip install sh
!pip install nlp
!pip install transformers
!pip install pytorch_lightning

     |████████████████████████████████| 40kB 1.7MB/s 
     |████████████████████████████████| 870kB 4.3MB/s 
     |████████████████████████████████| 63.8MB 43kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


     |████████████████████████████████| 675kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 11.1MB/s 
     |████████████████████████████████| 890kB 19.0MB/s 
     |████████████████████████████████| 3.8MB 30.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7a2d777684508f65ad79b0ae23df6d6e840f26e5583896e84aa18bce78555f73
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 256kB 4.6MB/s 
     |████████████████████████████████| 829kB 6.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=0b83bc08d1b24f613a835ee4ae8dde17e0c65ee840fb5c22df52bcc34eddc7da
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      

I have taken the reference from [hugging-face-nlp demo](https://github.com/yk/huggingface-nlp-demo/blob/master/demo.py) and modified accordingly to the quora question pair task.

As the dataset is pretty huge (4,00,000) pairs, running BERT on it will take 6-8hours of time. I took a subset of it and ran the code. If you want, you can modify the `percent` value accordingly.

### References:

- [Hugging-face-nlp-demo code](https://github.com/yk/huggingface-nlp-demo/blob/master/demo.py)
- [Hugging-face-nlp-demo video tutorial](https://www.youtube.com/watch?v=G3pOvrKkFuk&feature=youtu.be)


In [0]:
import sh

import nlp
import transformers
import torch as th
import pytorch_lightning as pl

In [0]:
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
debug = False
epochs = 10
batch_size = 8
lr = 1e-2
momentum = 0.9
model_type = 'bert-base-uncased'
seq_length = 100
percent = 20


sh.rm('-r', '-f', 'logs')
sh.mkdir('logs')

In [0]:
class QQPClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = transformers.BertForSequenceClassification.from_pretrained(model_type)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')
    
    def prepare_data(self, split="train"):
        tokenizer = transformers.BertTokenizer.from_pretrained(model_type)

        def _tokenize(x):
            encoded = tokenizer.batch_encode_plus(
                    x['question1'],
                    x['question2'],
                    max_length=seq_length, 
                    pad_to_max_length=True)
            x['input_ids'] = encoded['input_ids']
            x['token_type_ids'] = encoded['token_type_ids']
            return x

        def _prepare_ds(split):
            ds = nlp.load_dataset('glue', 'qqp', split=f'{split}[:{batch_size if debug else f"{percent}%"}]')
            ds = ds.map(_tokenize, batched=True)
            ds.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'label'])
            return ds

        self.train_ds, self.val_ds = map(_prepare_ds, ('train', 'validation'))

    def forward(self, input_ids, token_type_ids):
        mask = (input_ids != 0).float()
        logits, = self.model(input_ids, mask, token_type_ids)
        return logits

    def training_step(self, batch, batch_idx):
        logits = self.forward(batch['input_ids'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label']).mean()
        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        logits = self.forward(batch['input_ids'], batch['token_type_ids'])
        loss = self.loss(logits, batch['label'])
        acc = (logits.argmax(-1) == batch['label']).float()
        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        return {**out, 'log': out}

    def train_dataloader(self):
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=batch_size,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        return th.utils.data.DataLoader(
                self.val_ds,
                batch_size=batch_size,
                drop_last=False,
                shuffle=False,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=lr,
            momentum=momentum,
        )

In [0]:
model = QQPClassifier()
trainer = pl.Trainer(
    default_root_dir='logs',
    gpus=(1 if th.cuda.is_available() else 0),
    max_epochs=epochs,
    fast_dev_run=debug,
    logger=pl.loggers.TensorBoardLogger('logs/', name='qqp', version=0),
)
trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/qqp/1.0.0. Subsequent calls will reuse this data.


100%|██████████| 9/9 [00:02<00:00,  3.38it/s]

    | Name                                                   | Type                          | Params
-----------------------------------------------------------------------------------------------------
0   | model                                                  | BertForSequenceClassification | 109 M 
1   | model.bert                                             | BertModel                     | 109 M 
2   | model.bert.embeddings                                  | BertEmbeddings                | 23 M  
3   | model.bert.embeddings.word_embeddings                  | Embedding                     | 23 M  
4   | model.bert.embeddings.position_embeddings              | Embedding                     | 393 K 
5   | model.bert.embeddings.token_type_embeddings            | Embedding                     | 1 K   
6   | model.bert.embeddings.LayerNorm                        | LayerNorm                     | 1 K   
7   | model.bert.embeddings.dropout

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
